## SVM 분류 분석 실습

In [4]:
import pandas as pd
import seaborn as sns

from sklearn import svm

In [11]:
df = sns.load_dataset('titanic')

# NaN 값이 많은 deck열(변수) 삭제
# embarked와 embark_town 열(변수)는 의미가 동일하므로 embark_town 열 삭제
ndf = df.drop(['deck', 'embark_town'], axis=1)

#age변수의 값이 NaN인 행을 삭제
ndf = ndf.dropna(subset=['age'], how='any', axis=0)

#embared 열의 NaN값을 승선도시 중에서 가장 많이 출연한 데이터 값으로 치환하기
most_freq = ndf['embarked'].value_counts(dropna=True).idxmax()
ndf['embarked'].fillna(most_freq, inplace=True)

#분류 분석에서 사용할 변수 선택
X = ndf[['pclass', 'sex', 'age', 'sibsp', 'parch', 'embarked' ]]
Y = ndf['survived']

#범주형 데이터를 모델이 인식할 수 있는 숫자형 데이터로 변환
onehot_sex = pd.get_dummies(ndf['sex'])
ndf = pd.concat([ndf, onehot_sex], axis= 1)

onehot_embarked = pd.get_dummies(ndf['embarked'], prefix='town')
ndf = pd.concat([ndf, onehot_embarked], axis= 1)

X = ndf[['pclass', 'female', 'male', 'age', 'sibsp', 'parch', 'town_C','town_Q','town_S']]
Y = ndf['survived']

#분류 분석을 수행하려면 설명변수를 정규화 (평균 0, 표준편차1)
from sklearn import preprocessing 
X = preprocessing.StandardScaler().fit(X).transform(X)


# train data : test data 을 7:3으로 데이터 분리
from sklearn.model_selection import train_test_split
X_train, X_test, Y_train, Y_test = train_test_split(X, Y, test_size=0.3, random_state=10)

In [12]:
svm_model = svm.SVC(kernel= 'rbf')
svm_model.fit(X_train, Y_train)  #학습, 모델 생성

C:\Users\student\Anaconda3\lib\site-packages\sklearn\svm\base.py:193: FutureWarning: The default value of gamma will change from 'auto' to 'scale' in version 0.22 to account better for unscaled features. Set gamma explicitly to 'auto' or 'scale' to avoid this warning.
  "avoid this warning.", FutureWarning)


SVC(C=1.0, cache_size=200, class_weight=None, coef0=0.0,
    decision_function_shape='ovr', degree=3, gamma='auto_deprecated',
    kernel='rbf', max_iter=-1, probability=False, random_state=None,
    shrinking=True, tol=0.001, verbose=False)

In [13]:
y_predict = svm_model.predict(X_test)

In [14]:
print(y_predict[0:10])
print(Y_test.values[0:10])

[0 0 1 0 0 0 1 0 0 0]
[0 0 1 0 0 1 1 1 0 0]


In [15]:
from sklearn import metrics
svm_matrix = metrics.confusion_matrix(Y_test, y_predict)
print(svm_matrix)

[[120   5]
 [ 35  55]]


In [16]:
svm_report = metrics.classification_report(Y_test, y_predict)
print(svm_report)

              precision    recall  f1-score   support

           0       0.77      0.96      0.86       125
           1       0.92      0.61      0.73        90

    accuracy                           0.81       215
   macro avg       0.85      0.79      0.80       215
weighted avg       0.83      0.81      0.81       215



- SVM 은 데이터의 특성 개수가 작아도 복잡한 결정 경계를 만들어 주는 분류 방식입니다.
- 데이터 전처리(dummy변수, 정규화) 가 필요하며, 매개변수(하이퍼파마미터) 설정에 따라 분류 분석의 성능에 영향이 큼